In [36]:
import pandas as pd
import numpy as np
import cufflinks as cf
from sklearn.decomposition import PCA

In [55]:
dfmacro = pd.read_excel('macro.xlsx', index_col=0, parse_dates=True).sort_index().dropna()
dfret = dfmacro[['SPX Index', 'BCOM Index', 'DXY Curncy']].pct_change().dropna()
dfabs = dfmacro[['VIX Index', 'USGG10YR Index', 'USOHHYTO Index']].diff().dropna()

In [71]:
price_data = dfret.join(dfabs)
df_zsc = (price_data.sort_index() / (price_data.sort_index() ** 2).rolling(252).mean().apply(np.sqrt)).dropna()
myPCA = PCA()
myPCA.fit(df_zsc)
myPCA.explained_variance_ratio_
myPCA.fit_transform(df_zsc)


(2821, 6)

In [58]:
pd.DataFrame(index=df_zsc.columns, data=myPCA.components_[:4].T).iplot(kind='bar', barmode='stack', title='PCA Components', yTitle='PCA Component Value', xTitle='PCA Component', colorscale='spectral')

In [59]:
pd.DataFrame(index=df_zsc.columns, data=myPCA.components_[:4].T)

,0,1,2,3
SPX Index,-0.536527,-0.132501,0.425488,0.025104
BCOM Index,-0.255207,-0.446767,-0.521526,0.680358
DXY Curncy,-0.015059,0.752555,0.144425,0.603864
VIX Index,0.497764,0.086059,-0.456842,-0.112233
USGG10YR Index,-0.364282,0.444887,-0.484708,-0.231896
USOHHYTO Index,0.516052,-0.105704,0.287166,0.324744


In [50]:
myPCA = PCA()
myPCA.fit(df_zsc.tail(720))
pd.DataFrame(index=df_zsc.columns, data=myPCA.components_[:4].T)

,0,1,2,3
SPX Index,0.534222,-0.138997,-0.339184,0.204039
BCOM Index,0.282041,-0.114305,0.852518,0.417379
DXY Curncy,-0.330762,0.500139,-0.198532,0.772004
VIX Index,-0.434490,0.062046,0.278620,-0.227919
USGG10YR Index,0.150408,0.780237,0.194035,-0.368113
USOHHYTO Index,-0.560582,-0.323816,0.058937,0.026814


In [52]:
import numpy as np
from sklearn.datasets import load_digits
from scipy.spatial.distance import pdist
from sklearn.manifold.t_sne import _joint_probabilities
from scipy import linalg
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'sklearn.manifold.t_sne'

In [79]:
dffx = pd.read_excel('FX.xlsx', index_col=0, parse_dates=True).sort_index().dropna()
dffx.pct_change().dropna()
usdbrl = dffx['USDPLN Curncy'].pct_change().dropna()

In [80]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(price_data.loc[df_zsc.index], usdbrl.loc[df_zsc.index])
for i in range(len(reg.coef_)):
    print(price_data.columns[i], reg.coef_[i])
reg.score(price_data.loc[df_zsc.index], usdbrl.loc[df_zsc.index])

SPX Index -0.06740689972251893
BCOM Index 0.04696478090362878
DXY Curncy 1.2190577592412715
VIX Index 0.0002805812345131036
USGG10YR Index -0.002747446615199941
USOHHYTO Index 2.9670565691759798e-05


0.6392028991088928

In [81]:
reg = LinearRegression().fit(myPCA.fit_transform(df_zsc), usdbrl.tail(df_zsc.shape[0]))
reg.score(myPCA.fit_transform(df_zsc), usdbrl.tail(df_zsc.shape[0]))

0.6282823259483676